In [124]:
# Web Scraping (Google Shopping)
from serpapi import GoogleSearch
import pandas as pd
import numpy as np
import re

# Search queries
products = ["iPhone 11 64GB", "iPhone 11 128GB", "iPhone 11 256GB", "iPhone 11 pro 64GB", "iPhone 11 pro 256GB", "iPhone 11 pro 512GB", "iPhone 11 pro max 64GB", "iPhone 11 pro max 256GB", "iPhone 11 pro max 512GB"]

# Store all google shopping results
all = []

title = []
link = []
merchant = []
price = []
rating = []
delivery = []

# Trial version API, total plan searches limited to 100 times

for product in products:
    params = {"api_key": "a8adaefdaa487c187230af6503713403fd9786d7752c5766f220aa3a7c0e8b80","engine": "google","q": product,"gl": "au","hl": "en","tbm": "shop"}
    search = GoogleSearch(params)
    results = search.get_dict()
    all.append(results)
    

    for i in range(len(results["shopping_results"])):
        title.append(results["shopping_results"][i]["title"])
        link.append(results["shopping_results"][i]["link"])
        merchant.append(results["shopping_results"][i]["source"])
        price.append(results["shopping_results"][i]["price"])
        delivery.append(results["shopping_results"][i]["delivery"])

product_info_dict = {"Title": title, "ItemLink": link, "Merchant": merchant, "ItemPrice": price, "Delivery": delivery}
df = pd.DataFrame(product_info_dict)
df.to_csv("/Users/max/Desktop/iphone11_google_shopping_raw.csv")

https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search


In [95]:
# Data Cleaning

# ItemLink col, cut off google url
df["ItemLink"] = df["ItemLink"].str.replace("https://www.google.com/url\?url=", "")
# ItemPrice col, replace non-digit characters and change type to float
df["ItemPrice"] = df["ItemPrice"].str.replace("$", "").str.replace("\s.+x$", "").str.replace(",", "").astype(float)
# Delivery col, replace non-digit characters and change type to float
df["Delivery"] = df["Delivery"].str.replace("^F.+", "0").str.replace("$", "").str.replace("\s\w+y$", "").astype(float)
# PriceTotal col, selling price + shipping fee
df["PriceTotal"] = df["ItemPrice"] + df["Delivery"]
# Condition col, check refurb_keywords, if true, output "Refurbished", else, "New"
refurb_keywords = ["grade", "cert", "condition", "refurb", "fair", "as new", "like new", "excellent", "good", "cond", "face"]
df["Condition"] = np.where(df["Title"].str.contains("|".join(refurb_keywords), case=False), "Refubished", "New")
# Model col, check condition and do the matching with values
model_conditions = [(df["Title"].str.contains("max", case=False)), (df["Title"].str.contains("pro", case=False)), (~df["Title"].str.contains("pro", case=False))]
model_values = ["iPhone 11 pro max", "iPhone 11 pro", "iPhone 11"]
df["Model"] = np.select(model_conditions, model_values)
# Storage col, check condition and do the matching with values, finally replace unmatched value with "unspecified"
storage_conditions = [(df["Title"].str.contains("64", case=False)), (df["Title"].str.contains("128", case=False)), (df["Title"].str.contains("256", case=False)), (df["Title"].str.contains("512", case=False))]
storage_values = ["64GB", "128GB", "256GB", "512GB"]
df["Storage"] = np.select(storage_conditions, storage_values)
df["Storage"] = df["Storage"].str.replace("0", "unspecified")
df.to_csv("/Users/max/Desktop/iphone11_google_shopping_cleaned.csv")
df

,Title,ItemLink,Merchant,ItemPrice,Delivery,PriceTotal,Condition,Model,Storage
0,Apple iPhone 11 64GB Any Colour (Excellent Grade),https://www.kogan.com/au/buy/digitalstore-appl...,Kogan.com,565.00,0.00,565.00,Refubished,iPhone 11,64GB
1,Apple iPhone 11 64GB (White),https://www.harveynorman.com.au/apple-iphone-1...,Harvey Norman Australia,748.00,7.95,755.95,New,iPhone 11,64GB
2,Apple iPhone 11 - 4G smartphone - dual-SIM 64 ...,https://www.kogan.com/au/buy/eclipsetechnology...,Kogan.com,569.00,0.00,569.00,New,iPhone 11,64GB
3,"Apple iPhone 11 (64GB, Purple) - Afterpay & Zi...",https://www.kogan.com/au/buy/apple-iphone-11-6...,Kogan.com,829.00,10.99,839.99,New,iPhone 11,64GB
4,Apple iPhone 11 - (PRODUCT) RED - 4G smartphon...,https://www.dicksmith.com.au/da/buy/eclipsetec...,Dick Smith AU,569.00,0.00,569.00,New,iPhone 11 pro,64GB
...,...,...,...,...,...,...,...,...,...
559,Apple Iphone 11 Pro Max 512gb Midnight Green V...,https://www.ebay.com.au/itm/275219105762%3Fchn...,eBay.com.au - itsworthmore,839.59,0.00,839.59,Refubished,iPhone 11 pro max,512GB
560,Apple iPhone 11 Pro Max Nwhq2zp/A -512GB *Crac...,https://www.cashconverters.com.au/shop/phones-...,Cash Converters,999.00,22.45,1021.45,New,iPhone 11 pro max,512GB
561,Apple Iphone 11 Pro Max 64 /256 /512 Gb [all C...,https://www.ebay.com.au/itm/224675345578%3Fchn...,eBay.com.au,835.00,0.00,835.00,New,iPhone 11 pro max,64GB
562,"Apple iPhone 11 PRO MAX (512GB, Space Grey) - ...",https://www.becextech.com.au/catalog/iphone11p...,BecexTech Australia,1289.00,29.95,1318.95,Refubished,iPhone 11 pro max,512GB


In [157]:
# Read in eBay data frame, did some modification, make sure column name and data type are the same
ebay = pd.read_csv("/Users/max/Desktop/Iphone11_AUS.csv")

ebay.drop(["SubTitle", "Rating", "TrendingPrice", "Unnamed: 0", "Unnamed: 0.1"], axis=1, inplace=True)
ebay["Merchant"] = "eBay.com.au"
ebay.rename(columns={"shipping_cost": "Delivery", 'oldName2': 'newName2'}, inplace=True)
ebay["Delivery"] = ebay["Delivery"].astype(float)

refurb_keywords = ["used", "furbish", "crack", "imperfect", "fault", "grade", "cert", "condition", "refurb", "fair", "as new", "like new", "excellent", "good", "cond", "face"]
ebay["Condition"] = np.where(ebay["Title"].str.contains("|".join(refurb_keywords), case=False), "Refubished", "New")

model_conditions = [(ebay["Title"].str.contains("max", case=False)), (ebay["Title"].str.contains("pro", case=False)), (~ebay["Title"].str.contains("pro", case=False))]
model_values = ["iPhone 11 pro max", "iPhone 11 pro", "iPhone 11"]
ebay["Model"] = np.select(model_conditions, model_values)

storage_conditions = [(ebay["Title"].str.contains("64", case=False)), (ebay["Title"].str.contains("128", case=False)), (ebay["Title"].str.contains("256", case=False)), (ebay["Title"].str.contains("512", case=False))]
storage_values = ["64GB", "128GB", "256GB", "512GB"]
ebay["Storage"] = np.select(storage_conditions, storage_values)
ebay["Storage"] = ebay["Storage"].str.replace("0", "unspecified")

column_names = ['Title', 'ItemLink', 'Merchant', 'ItemPrice', 'Delivery', 'PriceTotal', 'Condition', 'Model', 'Storage']
ebay = ebay[column_names]
ebay.to_csv("/Users/max/Desktop/ebay.csv")

In [160]:
# Merge google and eBay data into 1 data frame for analysis
data1 = pd.read_csv("/Users/max/Desktop/iphone11_google_shopping_cleaned.csv")

data2 = pd.read_csv("/Users/max/Desktop/ebay.csv")

concate_data = pd.concat([data1,data2])
concate_data.to_csv("/Users/max/Desktop/final_data.csv")